In [7]:
from dotenv import load_dotenv
load_dotenv()


import json
from tqdm import tqdm
from openai import OpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored  

# [gpt-3.5-turbo-0125, gpt-4-0125-preview]
GPT_MODEL = "gpt-4-0125-preview"
client = OpenAI()

In [8]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        LOGGER.log_with_depth("Unable to generate ChatCompletion response")
        LOGGER.log_with_depth(f"Exception: {e}")
        return e


## decompose the problem into subproblems

In [9]:
system_prompt = '''
You are an expert capable of decomposing problems into subproblems as detailed subproblems as possible, ensuring each subproblem requires only one agent or a function to resolve. 
Allowing us to delegate each subproblem to the relevant agent or tools for a step-by-step resolution. 
This approach simplifies complex issues. We have three agents: the Safety Agent, the Efficacy Agent, and the Enrollment Agent. We have one function: predict_outcome.
For instance:
Question: How can we predict the outcome of a clinical trial?
Answer: <subproblem> To understand the safety of the drug, including toxicity and side effects, we need to consult the Safety Agent for safety information. </subproblem>
<subproblem> To assess the drug's efficacy against the diseases, it's necessary to ask the Efficacy Agent for information regarding the drug's effectiveness on the disease. </subproblem>
<subproblem> To determine if the clinical trial's eligibility criteria facilitate easy enrollment with enough patients, we should seek an enrollment analysis from the Enrollment Agent. </subproblem>
<subproblem> Use the predict_outcome function to combine the drug name, disease name, phase and eligibility criteria to predict the outcome of the clinical trial. </subproblem>
'''

In [10]:
user_prompt = open("demo_prompt.txt", "r").read()
LOGGER.log_with_depth(user_prompt)

# demo:
# duration: 577
# outcome: Terminated, Poor enrollment
# nctid: NCT00162214
# phase: Phase 1
# status: Terminated
# why_stop: 

I have designed a clinical trial and hope you can help me predict whether this trial can pass Phase I.
The following is my clinical trial:

disease: ['tumors']
drugs: ['dasatinib + ketoconazole']
phase: 1
criteria: 
        Inclusion Criteria:

          -  ECOG status 0-2

          -  Advanced or metastatic disease, unresponsive to standard treatment (or no standard
             treatment exists)

          -  Biopsy pretreatment

          -  Adequate bone marrow, liver and kidney function

        Exclusion Criteria:

          -  Serious cardiovascular disease

          -  Bleeding disorders

          -  Gastrointestinal (GI) tract disease

          -  Platelet inhibitors

          -  H2 blockers, proton pump inhibitors



In [11]:
response = chat_completion_request([{ "role": "system", "content": system_prompt }, { "role": "user", "content": user_prompt }])
LOGGER.log_with_depth(response.choices[0].message.content)


To predict whether this clinical trial can pass Phase I, we need to decompose the problem into specific subproblems that can be addressed by our agents and the function. The subproblems will focus on assessing the safety, efficacy, enrollment feasibility, and an overall prediction of the outcome based on specific trial details. Here's how we can break it down:

### Safety Assessment

<subproblem> Given the combination of drugs ('dasatinib + ketoconazole'), we need to consult the Safety Agent to identify potential toxicity, side effects, and drug interactions. This includes evaluating the safety concerns related to the combined use of these drugs in patients, especially those with advanced or metastatic tumors. </subproblem>

### Efficacy Assessment

<subproblem> We should ask the Efficacy Agent to provide data on the known effects of the drug combination 'dasatinib + ketoconazole' on tumors. This includes information on previous studies or any preliminary results that suggest the combi

In [ ]:
import re
string = response.choices[0].message.content

subproblems = re.findall(r"<subproblem>(.*?)</subproblem>", string)
subproblems = [subproblem.strip() for subproblem in subproblems]

LOGGER.log_with_depth(subproblems)


['Consult the Safety Agent to evaluate the safety profile of the combination therapy with dasatinib and ketoconazole, focusing on potential toxicities, side effects, and interactions that may pose risks, particularly in the context of tumors. This will include assessing risks related to cardiovascular diseases, bleeding disorders, GI tract diseases, and effects on platelet function, as these areas are highlighted in the exclusion criteria.', 'Inquire the Efficacy Agent about the expected efficacy of dasatinib combined with ketoconazole in treating advanced or metastatic tumors unresponsive to standard treatments. This inquiry should consider the mechanism of action of both drugs, historical data on their use (individually or in combination) in similar or different tumors, and any preliminary results that may indicate a positive outcome.', 'Request an analysis from the Enrollment Agent on the feasibility of enrolling sufficient patients based on the specified inclusion and exclusion cri

## Solve the subproblems

In [2]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "safety_agent",
            "description": "To understand the safety of the drug, including toxicity and side effects, consult the Safety Agent for safety information. Given drug name, return the safety information of the drug, e.g. ADMET, drug introduction, toxicity and side effects etc.",
            "parameters": {
                "type": "object",
                "properties": {
                    "drug_name": {
                        "type": "string",
                        "description": "The drug name",
                    }
                },
                "required": ["drug_name"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "efficacy_agent",
            "description": "To assess the drug's efficacy against the diseases, ask the Efficacy Agent for information regarding the drug's effectiveness on the disease. Given drug name and disease name, return the drug introduction, disease introduction, and the path between drug and disease in the hetionet knowledge graph etc.",
            "parameters": {
                "type": "object",
                "properties": {
                    "drug_name": {
                        "type": "string",
                        "description": "The drug name",
                    },
                    "disease_name": {
                        "type": "string",
                        "description": "The disease name",
                    }
                },
                "required": ["drug_name", "disease_name"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "enrollment_agent",
            "description": "To determine if the clinical trial's eligibility criteria facilitate easy enrollment with enough patients. Given eligibility criteria, return the clinical trial will be poor enrollment, good enrollment, or excellent enrollment.",
            "parameters": {
                "type": "object",
                "properties": {
                    "eligibility_criteria": {
                        "type": "string",
                        "description": "eligibility criteria, contains including criteria and excluding criteria",
                    }
                },
                "required": ["eligibility_criteria"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "predict_outcome",
            "description": "Given the drug name, phase, disease name, eligibility criteria, predict the outcome of the clinical trial. Here, the trial outcome refers to a binary success indicator whether the trial is completed to meet their primary endpoints. ",
            "parameters": {
                "type": "object",
                "properties": {
                    "drug_name": {
                        "type": "string",
                        "description": "The drug name",
                    },
                    "phase": {
                        "type": "string",
                        "description": "The phase of the clinical trial",
                    },
                    "disease_name": {
                        "type": "string",
                        "description": "The disease name",
                    },
                    "eligibility_criteria": {
                        "type": "string",
                        "description": "eligibility criteria, contains including criteria and excluding criteria",
                    }
                },
            }
        }
    },
]



In [5]:
# tools

func_content = [
    {'function_name': tool['function']['name'], 'description': tool['function']['description']} for tool in tools
]

LOGGER.log_with_depth(func_content)
LOGGER.log_with_depth(json.dumps(func_content, indent=4))

[{'function_name': 'safety_agent', 'description': 'To understand the safety of the drug, including toxicity and side effects, consult the Safety Agent for safety information. Given drug name, return the safety information of the drug, e.g. ADMET, drug introduction, toxicity and side effects etc.'}, {'function_name': 'efficacy_agent', 'description': "To assess the drug's efficacy against the diseases, ask the Efficacy Agent for information regarding the drug's effectiveness on the disease. Given drug name and disease name, return the drug introduction, disease introduction, and the path between drug and disease in the hetionet knowledge graph etc."}, {'function_name': 'enrollment_agent', 'description': "To determine if the clinical trial's eligibility criteria facilitate easy enrollment with enough patients. Given eligibility criteria, return the clinical trial will be poor enrollment, good enrollment, or excellent enrollment."}, {'function_name': 'predict_outcome', 'description': 'Give

In [28]:
def predict_outcome(drug_name, phase, disease_name, eligibility_criteria):
    
    return 0.7



In [29]:
system_prompt = '''
You are an expert capable of solve the subproblems which decomposed from the original problem.
You can use the tools to ask agents or tools to solve the subproblems step by step.
This approach simplifies complex issues. We have three agents: the Safety Agent, the Efficacy Agent, and the Enrollment Agent. And we have one function: predict_outcome.
'''

In [30]:
def exec_func(choices):
    results = []

    for choice in choices:
        if choice.finish_reason == "tool_calls":
            tool_calls = choice.message.tool_calls
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                arguments = tool_call.function.arguments
                arguments = json.loads(arguments)

                result =  eval(f"{function_name}(**{arguments})")

                results.append((
                    f"The function {function_name} is called and the result is {result}",
                    result,
                ))

    return results

exec_func(response.choices)

[]

## Define the Agents

### Safety Agent

In [31]:
def get_ADMET_property(drug_name):

    admet_json = {
        "Absorption": "High",
        "Distribution": "Low",
        "Metabolism": "Medium",
        "Excretion": "High",
        "Toxicity": "Low"
    }

    return admet_json

def get_SMILES(drug_name):
    return "CC1=CC(=C(C=C1)C(=O)O)O"

def retrieval_drugbank(drug_name):

    drugbank_info = ''' 
        Dasatinib is a tyrosine kinase inhibitor used in the treatment of certain types of chronic myeloid leukemia (CML) and acute lymphoblastic leukemia (ALL) that are Philadelphia chromosome-positive. It works by blocking the action of an abnormal protein that signals cancer cells to multiply. This helps to stop the spread of cancer cells.
        Ketoconazole is an antifungal agent that can also inhibit certain cytochrome P450 enzymes, particularly CYP3A4, which are involved in the metabolism of many drugs.
        When dasatinib and ketoconazole are taken together, there can be an interaction because ketoconazole can inhibit the metabolism of dasatinib. This inhibition can lead to increased levels of dasatinib in the blood, potentially leading to an increase in side effects or toxicity. Side effects of dasatinib include, but are not limited to, myelosuppression, bleeding, fluid retention, and cardiac events.
        Patients taking dasatinib in combination with ketoconazole or other potent CYP3A4 inhibitors should be monitored closely for adverse reactions associated with high dasatinib levels, and dose adjustments of dasatinib may be necessary.
    '''

    return drugbank_info

def safety_agent(drug_name):
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_ADMET_property",
                "description": "Provided with a drug's name, the model offers insights into its Absorption, Distribution, Metabolism, Excretion, and Toxicity (ADMET) characteristics.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "drug_name": {
                            "type": "string",
                            "description": "The drug name",
                        }
                    },
                    "required": ["drug_name"],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_SMILES",
                "description": "Given a drug's name, the model returns its Simplified Molecular Input Line Entry System (SMILES) notation.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "drug_name": {
                            "type": "string",
                            "description": "The drug name",
                        }
                    },
                    "required": ["drug_name"],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "retrieval_drugbank",
                "description": "Given a drug's name, the model retrieves its information from DrugBank database.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "drug_name": {
                            "type": "string",
                            "description": "The drug name",
                        }
                    },
                    "required": ["drug_name"],
                },
            }
        }
    ]
    
    messages = []

    system_prompt = ''' 
    As a drug safety expert, you have the capability to assess a drug's safety by examining its toxicity and side effects. 
    You can use tools to obtain the ADMET properties of the drug, access information from DrugBank, and retrieve the drug's SMILES notation. 
    Moreover, you can offer insights into the drug based on your expertise. 
    '''

    messages.append({ "role": "system", "content": system_prompt})

    messages.append(
    { "role": "user", "content": '''
        To begin, you must break down the user's problem into smaller, manageable subproblems in <subproblem></subproblem>. Ensuring each subproblem requires only a function to resolve. For instance:
        Question: How can I evaluate the safety of the drug aspirin?
        Answer: <subproblem> To understand the drug's structure, we need to obtain the SMILES notation of the drug. </subproblem>
        <subproblem>To assess the drug's safety profile, we should retrieve information from the DrugBank database. </subproblem>
        <subproblem>To evaluate the drug's ADMET properties, we need to obtain the Absorption, Distribution, Metabolism, Excretion, and Toxicity characteristics of the drug by calling the get_ADMET_property function.</subproblem>
        <subproblem>Provide the insight into drug safety, including toxicity and side effects, based on your knowledge.</subproblem>
    '''}
    )

    messages.append({ "role": "user", "content": f"Please you help me to evaluate the safety of the drug: {drug_name}"})

    response = chat_completion_request(messages)
    messages.append({ "role": "assistant", "content": response.choices[0].message.content})

    subproblems_str = messages[-1]["content"]

    subproblems = re.findall(r"<subproblem>(.*?)</subproblem>", subproblems_str)
    subproblems = [subproblem.strip() for subproblem in subproblems]

    problem_results = []
    for sub_problem in tqdm(subproblems):
        response = chat_completion_request(
            [{ "role": "system", "content": system_prompt }, {
                "role": "user", "content": f"The original user problem is: {user_prompt}"
            }, { "role": "user", "content": sub_problem }],
            tools=tools)
        
        sub_results = [r[0] for r in exec_func(response.choices)]
        
        problem_results.extend(sub_results)

    return problem_results


In [32]:
agent_results = safety_agent("dasatinib + ketoconazole")

LOGGER.log_with_depth(agent_results)

100%|██████████| 5/5 [00:20<00:00,  4.14s/it]

['The function get_SMILES is called and the result is CC1=CC(=C(C=C1)C(=O)O)O', 'The function get_SMILES is called and the result is CC1=CC(=C(C=C1)C(=O)O)O', 'The function retrieval_drugbank is called and the result is  \n        Dasatinib is a tyrosine kinase inhibitor used in the treatment of certain types of chronic myeloid leukemia (CML) and acute lymphoblastic leukemia (ALL) that are Philadelphia chromosome-positive. It works by blocking the action of an abnormal protein that signals cancer cells to multiply. This helps to stop the spread of cancer cells.\n        Ketoconazole is an antifungal agent that can also inhibit certain cytochrome P450 enzymes, particularly CYP3A4, which are involved in the metabolism of many drugs.\n        When dasatinib and ketoconazole are taken together, there can be an interaction because ketoconazole can inhibit the metabolism of dasatinib. This inhibition can lead to increased levels of dasatinib in the blood, potentially leading to an increase i

### Efficacy Agent

In [33]:
def get_disease_icdcode(disease_name):
    return "E11.9"

def get_disease_risk(disease_name):
    return 0.6

def retrieval_drugbank(drug_name):
    drugbank_info = ''' 
        Dasatinib is a tyrosine kinase inhibitor used in the treatment of certain types of chronic myeloid leukemia (CML) and acute lymphoblastic leukemia (ALL) that are Philadelphia chromosome-positive. It works by blocking the action of an abnormal protein that signals cancer cells to multiply. This helps to stop the spread of cancer cells.
        Ketoconazole is an antifungal agent that can also inhibit certain cytochrome P450 enzymes, particularly CYP3A4, which are involved in the metabolism of many drugs.
        When dasatinib and ketoconazole are taken together, there can be an interaction because ketoconazole can inhibit the metabolism of dasatinib. This inhibition can lead to increased levels of dasatinib in the blood, potentially leading to an increase in side effects or toxicity. Side effects of dasatinib include, but are not limited to, myelosuppression, bleeding, fluid retention, and cardiac events.
        Patients taking dasatinib in combination with ketoconazole or other potent CYP3A4 inhibitors should be monitored closely for adverse reactions associated with high dasatinib levels, and dose adjustments of dasatinib may be necessary.
    '''

    return drugbank_info

def retrieval_hetionet(drug_name, disease_name):

    kg_path = [
        ("Dasatinib", "Inhibits", "SRC Family Kinases"), 
        ("SRC Family Kinases", "Involved in", "Adhesion, Migration, and Angiogenesis"), 
        ("Adhesion, Migration, and Angiogenesis", "Facilitate", "Tumor Metastasis and Growth"), 
        ("Dasatinib", "Reduces/Inhibits", "Tumor Metastasis and Growth"),

        ("Dasatinib", "Inhibits", "PDGFR"), 
        ("PDGFR", "Participates in", "Cell Growth and Survival Pathways"), 
        ("Cell Growth and Survival Pathways", "Lead to", "Enhanced Cellular Proliferation and Survival"), 
        ("Enhanced Cellular Proliferation and Survival", "Promotes", "Tumor Progression"), 
        ("Dasatinib", "Reduces/Inhibits", "Tumor Progression"),

        ("Dasatinib", "Inhibits", "EPHA2"), 
        ("EPHA2", "Involved in", "Oncogenic Signaling Pathways"), 
        ("Oncogenic Signaling Pathways", "Contribute to", "Tumor Cell Migration and Invasion"), 
        ("Tumor Cell Migration and Invasion", "Facilitates", "Tumor Metastasis"), 
        ("Dasatinib", "Reduces/Inhibits", "Tumor Metastasis")
    ]

    return kg_path

def get_SMILES(drug_name):
    return "CC1=CC(=C(C=C1)C(=O)O)"

def efficacy_agent(drug_name, disease_name):


    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_disease_icdcode",
                "description": ''' 
                Given the name of a disease, the model returns the ICD code for the disease. 
                The International Classification of Diseases (ICD) code is a global system used for classifying health conditions and diseases, managed by the WHO. 
                It standardizes health data for clinical, epidemiological, and administrative purposes, facilitating worldwide health information exchange.
                ''',
                "parameters": {
                    "type": "object",
                    "properties": {
                        "disease_name": {
                            "type": "string",
                            "description": "The disease name",
                        }
                    },
                    "required": ["disease_name"],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_disease_risk",
                "description": "Given the name of a disease, the model returns the the historical success rate of the target disease in clinical trials.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "disease_name": {
                            "type": "string",
                            "description": "The disease name",
                        }
                    },
                    "required": ["disease_name"],
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "retrieval_drugbank",
                "description": "Given a drug's name, the model retrieves its information from DrugBank database.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "drug_name": {
                            "type": "string",
                            "description": "The drug name",
                        }
                    },
                    "required": ["drug_name"],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "retrieval_hetionet",
                "description": '''
                Given the names of a drug and a disease, the model retrieves the path connecting the drug to the disease from the Hetionet Knowledge Graph. 
                Hetionet is a comprehensive knowledge graph that integrates diverse biological information by connecting genes, diseases, compounds, and more into an interoperable framework. 
                It structures real-world biomedical data into a network, facilitating advanced analysis and discovery of new insights into disease mechanisms, drug repurposing, and the genetic underpinnings of health and disease.
                ''',
                "parameters": {
                    "type": "object",
                    "properties": {
                        "drug_name": {
                            "type": "string",
                            "description": "The drug name",
                        },
                        "disease_name": {
                            "type": "string",
                            "description": "The disease name",
                        }
                    },
                    "required": ["drug_name", "disease_name"],
                },
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_SMILES",
                "description": "Given a drug's name, the model returns its Simplified Molecular Input Line Entry System (SMILES) notation.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "drug_name": {
                            "type": "string",
                            "description": "The drug name",
                        }
                    },
                    "required": ["drug_name"],
                },
            }
        },
    ]


    

    messages = []

    system_prompt = ''' 
        As an efficacy expert, you have the capability to assess a drug's efficacy against diseases by examining its effectiveness on the disease. 
        You can use tools to obtain information on the drug, the disease, the disease's historical success rate, and the connection between the drug and disease in the Hetionet knowledge graph, among other things.
        Furthermore, you can offer insights into the drug and disease based on your expertise.
    '''

    messages.append({ "role": "system", "content": system_prompt})

    messages.append(
    { "role": "user", "content": '''
        To begin, you must break down the user's problem into smaller, manageable subproblems in <subproblem></subproblem>. Ensuring each subproblem requires only a function to resolve. For instance:
        Question: How can I evaluate the efficacy of the drug aspirin on the disease diabetes?
        Answer:
        <subproblem> To understand the drug's structure, we need to obtain the SMILES notation of the drug. </subproblem>
        <subproblem> To assess the drug's efficacy against the disease, we should retrieve information from the DrugBank database. </subproblem>
        <subproblem> To determine the historical success rate of the target disease in clinical trials, we need to call the get_disease_risk function. </subproblem>
        <subproblem> To retrieve the ICD code for the disease, we need to call the get_disease_icdcode function. </subproblem>
        <subproblem> To retrieve the path connecting the drug to the disease from the Hetionet Knowledge Graph, we need to call the retrieval_hetionet function. </subproblem>
        <subproblem> Provide insights into the drug and disease based on your expertise and do not call any external tools. </subproblem>
    '''}
    )

    messages.append({ "role": "user", "content": f"Please you help me to evaluate the efficacy of the drug {drug_name} on the disease {disease_name}"})

    response = chat_completion_request(messages)
    messages.append({ "role": "assistant", "content": response.choices[0].message.content})


    subproblems_str = messages[-1]["content"]

    subproblems = re.findall(r"<subproblem>(.*?)</subproblem>", subproblems_str)
    subproblems = [subproblem.strip() for subproblem in subproblems]

    problem_results = []
    for sub_problem in tqdm(subproblems):
        response = chat_completion_request(
            [{ "role": "system", "content": system_prompt }, {
                "role": "user", "content": f"The original user problem is: {user_prompt}"
            }, { "role": "user", "content": sub_problem }],
            tools=tools)
        
        sub_results = [r[0] for r in exec_func(response.choices)]
        
        problem_results.extend(sub_results)

    return problem_results


In [34]:
agent_results = efficacy_agent("dasatinib + ketoconazole", "tumors")

LOGGER.log_with_depth(agent_results)

100%|██████████| 10/10 [00:53<00:00,  5.39s/it]

['The function get_SMILES is called and the result is CC1=CC(=C(C=C1)C(=O)O)', 'The function get_SMILES is called and the result is CC1=CC(=C(C=C1)C(=O)O)', 'The function retrieval_drugbank is called and the result is  \n        Dasatinib is a tyrosine kinase inhibitor used in the treatment of certain types of chronic myeloid leukemia (CML) and acute lymphoblastic leukemia (ALL) that are Philadelphia chromosome-positive. It works by blocking the action of an abnormal protein that signals cancer cells to multiply. This helps to stop the spread of cancer cells.\n        Ketoconazole is an antifungal agent that can also inhibit certain cytochrome P450 enzymes, particularly CYP3A4, which are involved in the metabolism of many drugs.\n        When dasatinib and ketoconazole are taken together, there can be an interaction because ketoconazole can inhibit the metabolism of dasatinib. This inhibition can lead to increased levels of dasatinib in the blood, potentially leading to an increase in 

### Enrollment Agent

In [35]:
def enrollment_difficulty_model(eligibility_criteria):
    return 0.2


def enrollment_agent(eligibility_criteria):
    tools = [
        {
            "type": "function",
            "function": {
                "name": "enrollment_difficulty_model",
                "description": "Given the eligibility criteria of a clinical trial, the model predicts the enrollment difficulty of the trial.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "eligibility_criteria": {
                            "type": "string",
                            "description": "The eligibility criteria of the clinical trial",
                        }
                    },
                    "required": ["eligibility_criteria"],
                },
            }
        }
    ]

    messages = []

    system_prompt = ''' 
        As an enrollment expert, you have the capability to predict the enrollment difficulty of a clinical trial based on its eligibility criteria. 
        You can use tools to predict the enrollment difficulty of the trial. 
    '''

    messages.append({ "role": "system", "content": system_prompt})
    messages.append({ "role": "user", "content": f"Please you help me to evaluate the enrollment difficulty of the clinical trial with eligibility criteria."})
    messages.append({ "role": "user", "content": user_prompt})

    response = chat_completion_request(messages, tools=tools)

    problem_results = [r[0] for r in exec_func(response.choices)]

    return problem_results


In [36]:
agent_results = enrollment_agent("Inclusion Criteria: Patients with type 2 diabetes; Exclusion Criteria: Patients with type 1 diabetes")

LOGGER.log_with_depth(agent_results)

['The function enrollment_difficulty_model is called and the result is 0.2']


## Solve the subproblems step by step

In [37]:
problem_results = []

for sub_problem in subproblems:
    response = chat_completion_request(
        [{ "role": "system", "content": system_prompt }, {
            "role": "user", "content": f"The original user problem is: {user_prompt}"
        }, { "role": "user", "content": sub_problem }],
        tools=tools)
    
    sub_results = [r[0] for r in exec_func(response.choices)]

    problem_results.extend(sub_results)


100%|██████████| 6/6 [00:38<00:00,  6.45s/it]


In [38]:
LOGGER.log_with_depth(problem_results)

['The function safety_agent is called and the result is [\'The function get_SMILES is called and the result is CC1=CC(=C(C=C1)C(=O)O)\', \'The function get_SMILES is called and the result is CC1=CC(=C(C=C1)C(=O)O)\', \'The function retrieval_drugbank is called and the result is  \\n        Dasatinib is a tyrosine kinase inhibitor used in the treatment of certain types of chronic myeloid leukemia (CML) and acute lymphoblastic leukemia (ALL) that are Philadelphia chromosome-positive. It works by blocking the action of an abnormal protein that signals cancer cells to multiply. This helps to stop the spread of cancer cells.\\n        Ketoconazole is an antifungal agent that can also inhibit certain cytochrome P450 enzymes, particularly CYP3A4, which are involved in the metabolism of many drugs.\\n        When dasatinib and ketoconazole are taken together, there can be an interaction because ketoconazole can inhibit the metabolism of dasatinib. This inhibition can lead to increased levels o

In [39]:
messages = []

system_prompt = ''' 
    You are an expert in clinical trials. Based on the subproblems have solved, please solve the user's problem and provide the reason.
    Firstly, please give the final result of the user's problem.
    Secondly, please provide the reason step by step for the result.
'''

messages.append({ "role": "system", "content": system_prompt})
messages.append({ "role": "user", "content": f"The original user problem is: {user_prompt}"})
messages.append({ "role": "user", "content": f"The subproblems have solved are: {problem_results}"})
messages.append({ "role": "user", "content": "Please solve the user's problem and provide the reason."})

response = chat_completion_request(messages)

LOGGER.log_with_depth(response.choices[0].message.content)


Final Result: Based on the analysis, it is predicted that the clinical trial could likely pass Phase I.

Reasoning Step by Step:

1. **Drug Combination and Action Mechanism**: The trial involves a combination of dasatinib and ketoconazole for treating tumors. Dasatinib is a tyrosine kinase inhibitor effective against certain types of chronic myeloid leukemia (CML) and acute lymphoblastic leukemia (ALL), blocking abnormal protein signals that prompt cancer cells to multiply. Ketoconazole, an antifungal agent, can inhibit CYP3A4 enzymes that metabolize many drugs, including dasatinib. This combination suggests a careful selection aimed at enhancing anti-cancer efficacy, but also highlights potential for increased toxicity due to drug interaction.

2. **Pharmacokinetics and Potential Drug Interaction**: The information provided outlines that when dasatinib and ketoconazole are taken together, there can be significant drug-drug interaction leading to increased dasatinib levels due to inhib